<a href="https://colab.research.google.com/github/DanyalKanzanAhmad/Project-02-LangChain-RAG-Project/blob/main/Project_02_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community pypdf -q langchain_google_genai  langchain_chroma -q

In [ ]:
!pip install  --upgrade -q google-auth google-auth-oauthlib google-auth-httplib2  google-cloud


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/Digital Products Ideas.pdf")
data = loader.load()
len(data)

5

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
len(docs)

5

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gen-lang-client-0213865457-0d4aa550314c.json"


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GOOGLE_API_KEY)


In [ ]:
from langchain_chroma import Chroma
from langchain.docstore.document import Document

# Ensure 'docs' is not empty and contains valid documents
if not docs:
    print("Warning: 'docs' is empty. Please check your document loading and splitting process.")

# Iterate through documents and generate embeddings individually
embeddings = [embedding_model.embed_query(doc.page_content) for doc in docs]
# Create a list of Document objects with embeddings
new_docs = [Document(page_content=doc.page_content, metadata=doc.metadata, embedding=emb) for doc, emb in zip(docs, embeddings)]


vectorstore = Chroma.from_documents(documents=new_docs, embedding=embedding_model, persist_directory="./chroma_db") # Pass embeddings directly to Chroma
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [ ]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=docs,embedding = embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , temperature = 0.2 , max_tokens=1000)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
response = rag_chain.invoke({"input" : "what is multi attention"})
print(response["answer"])